#**Funções de pré-processamento**
Nome: Julia da Silva Godinho (juliagodinho08@gmail.com)


---


Nesse notebook são definidas as funções:

* `preprocessamento`  pre-processamento dos dados
* `regressao`  preenchimento de falhas
* `KNN`  preenchimento de falhas
* `criando_dados_modelo` para crianção do df com variveis do modelo



## Abrindo os arquivos 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
#!pip install icecream
#sfrom icecream import ic

#Abrindo o arquivo 
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
galdiArquivo = '/content/drive/My Drive/IC - Aprendizado de Máquina/Arquivos CSV/chuvas_C_02242004.csv'
oratArquivo = '/content/drive/My Drive/IC - Aprendizado de Máquina/Arquivos CSV/Pluvio 2241004 Fazenda Oratorio.csv'
pillerArquivo = "/content/drive/MyDrive/IC - Aprendizado de Máquina/Arquivos CSV/chuvas_C_02242003.csv"
PillerFluArquivo = '/content/drive/MyDrive/IC - Aprendizado de Máquina/Arquivos CSV/vazoes_C_59135000.csv'

data_galdi = pd.read_csv(galdiArquivo,index_col=False,skiprows=12,encoding='latin-1',sep=';')
data_orat = pd.read_csv(oratArquivo,index_col=False,skiprows=12,encoding='latin-1',sep=';')
data_piller = pd.read_csv(pillerArquivo,index_col=False,skiprows=12,encoding='latin-1',sep=';')
data_flu_piller = pd.read_csv(PillerFluArquivo,index_col=False,skiprows=12,encoding='latin-1',sep=';')

##**Função `preprocessamento` - Organização dos Dados**

**Parametros**:
* `df` dataframe 
* `colunas_dados` as colunas de interesse que serão mantidas após os tratamentos
* `coluna_index` o nome da coluna que possui as datas
* `nome_estacao` o nome da estação

Essa função retorna um dataframe com dados mensais:

* filtadro com os colunas de interesse
* com index em datetime
* com uma observação por mês
* dados convertidos para float 
* sem datas duplicadas


In [ ]:
def preprocessamento (df, colunas_dados, coluna_index, nome_estacao, 
                      dados = "consistidos", freq = "mensal", 
                      inicio = None, fim = None,
                      eto = False):
  
  """Retorna o dataframe com index ordenado, em formato datetime e com os dados em float. 
    Para observações diárias, retorna com uma observação por linha. 

------
  Parâmetros
   - `df`: dataframe com os dados brutos
   - `colunas_dados`: conjunto de colunas que serão mantidas no df
   - `coluna_index`: nome da coluna que vivará o index
   - `nome_estacao`: nome da estacao 
   - `dados`: escolher entre manter os dados 'brutos' os ou dados 'consistidos' 
   - `freq`: escolher entre duas frequências 'diaria' ou 'mensal'
   - `inicio`: dada de inicio da série temporal (Opcional)
   - `fim`: data final da série temporal (Opcional)
   - `eto`: True se o dataframe for para calculo da ETo
   """

  df2 = df.loc[:]

  # 1 - RETIRADA DE DADOS DUPLICADOS
  # Mantem a ultima ocorrência que representam as observações consistidas
  if dados == "brutos": #os dados brutos são as primeira ocorrências
    df2 = df2.drop_duplicates([coluna_index], keep='first')
  else: #os dados consistidos são as segundas ocorrências
    df2 = df2.drop_duplicates([coluna_index], keep='last')

  # 2 - ALTERAÇÃO E ORDENAÇÃO DO INDEX
  # Ordenação crescente
  df2[coluna_index] = pd.to_datetime(df2[coluna_index],format='%d/%m/%Y')
  df2 = df2.set_index(coluna_index)
  df2 = df2.sort_index(axis=0,ascending=True)
  
  # 3 - CONVERSÃO DOS DADOS PARA FLOAT
  for coluna in colunas_dados:
    df2[coluna] = df2[coluna].apply(lambda x: str(x).replace(',','.'))
    df2[coluna] = df2[coluna].astype(float)

  # 4 - GARANTINDO A FREQUENCIA DOS DADOS

  # 4.1 ESTAÇÕES METEREOLOGICAS - EVAPOTRANSPIRAÇAO
  if eto:  

    df3 = pd.DataFrame(columns = colunas_dados)

    if freq == 'diaria': f = 'D'
    else: f = 'M'

    for coluna in colunas_dados:
      if coluna == 'PRECIPITACAO TOTAL, DIARIO (AUT)(mm)':
        df3[coluna] = df2[coluna].resample(f).sum()
        #print('oi')
      else: 
        #print('Falso')
        df3[coluna] = df2[coluna].resample(f).mean()

  # 4.2 ESTAÇOES FLUVIOMETRICAS E PLUVIOMETRICAS
  else: 
    
    # Caso a data não exista, será criada uma nova linha com dados nulos 
    df_mensal = df2.resample('M').mean()
    
    # 4.2.1 DADOS DIARIOS - TRANSFORMANDO LINHA EM COLUNA
    if freq == "diaria":
      # Garantindo que tenha uma linha por dia
      df_diario = df2.resample('D').mean()

      # Criando um dataframe com somente uma coluna para dados diários
      indices = df_diario.index
      df3 = pd.DataFrame(data = None, index = indices, columns = ['Dados'])

      for dados_mensais in df_mensal.index: 
        data_obs = dados_mensais.strftime("%Y-%m-%d")
        num_dias = dados_mensais.daysinmonth
    
        for dados_diarios, dia in zip(colunas_dados, range(1,num_dias+1)):
          # Acessando os valores diários
          value = df_mensal.loc[data_obs,dados_diarios]

          # Armazenando o valor na data correspodente
          data = dados_mensais.replace(day = dia)
          df3.loc[data,'Dados'] = value

    # 4.2.2. DADOS MENSAIS
    else:  
      df3 = df_mensal[colunas_dados]
  
  # Selecionando o intervalo temporal dos dados 
  if inicio != None and fim != None:
    df3 = df3[inicio:fim]
  
  # 5 - RETORNA A BASE HISTORICA DA ESTACAO E QUANTIDADE DE FALHAS
  #falhas = df2[colunas_dados].isnull().sum().sum()
  observacoes = df3.shape[0]
  data_inicio = df3.index[0].strftime("%d-%m-%Y") 
  data_fim = df3.index[observacoes-1].strftime("%d-%m-%Y")
  falhas = df3.isnull().sum().sum()
  print('%s | Observações: %i | Freq: %s | Período: %s a %s | Falhas: %i' %(nome_estacao,
                                                              observacoes,
                                                              str(freq),
                                                              data_inicio,
                                                              data_fim,
                                                              falhas))

  return (df3)

### Teste

In [ ]:
# Macae_Arquivo = '/content/drive/My Drive/IC - Aprendizado de Máquina/Climático/dados_A608_D_2006-09-21_2020-04-01.csv'
# df_macae = pd.read_csv(Macae_Arquivo,index_col=False,skiprows=10,encoding='latin-1',sep=';') 

In [ ]:
# # As colunas de interesse 
# colunas_macae = ['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)','TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)',
#            'TEMPERATURA MEDIA, DIARIA (AUT)(Â°C)','TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)',
#            'UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%)','VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s)']

# # Mudando o formato da datas de medição
# # para ficarem compatíveis com as previstas na função de preprocessamento
# df_macae['Data Medicao'] = pd.to_datetime(df_macae['Data Medicao'],format='%Y/%m/%d')

# # Fazendo o preprocessamento dos dados
# data_macae = preprocessamento(df_macae,colunas_macae,'Data Medicao',
#                               'Automática Macae', inicio = '10-2006',
#                               fim = '03-2020', eto = True)

# data_macae

In [ ]:
# col = data_piller.columns[13:44] #para dados diários
# col2 = ['Total','Maxima','NumDiasDeChuva'] #para dados mensais
# col_flu = data_flu_piller.columns[16:47]

# piller_diario = preprocessamento(data_piller, colunas_dados = col,
#                              coluna_index = "Data",
#                              nome_estacao = "Piller", freq = 'diaria', 
#                              inicio = '1997', fim = '1998')

# piller_flu_diario = preprocessamento(data_flu_piller, colunas_dados = col_flu,
#                              coluna_index = "Data",
#                              nome_estacao = "Piller", freq = 'diaria', 
#                              inicio = '1997', fim = '1998')


# piller_mensal = preprocessamento(data_piller, col2, coluna_index = "Data",
#                                 nome_estacao = "Piller")

# galdi_diario = preprocessamento(data_galdi, col, "Data", "Galdinópolis",
#                                 freq = 'diaria', inicio = '1997', fim = '1998')

A estação Piller tem 730 observações com frequencia diaria de 01-01-1997 até 31-12-1998
A estação Piller tem 730 observações com frequencia diaria de 01-01-1997 até 31-12-1998
A estação Piller tem 837 observações com frequencia mensal de 31-08-1950 até 30-04-2020
A estação Galdinópolis tem 730 observações com frequencia diaria de 01-01-1997 até 31-12-1998


##**Função `regressao` - Preenchimento de Falhas**

**Parametros**:
* `df_y` dataframe com as variaveis alvo 
* `df_x` dataframe com variaveis preditoras
* `colunas_y` colunas com as falhas 
* `colunas_x` colunas da variavel preditora

A função retorna o dataframe `df_y` com novas colunas em que as falhas foram preenchidas. 

Antes de serem usadas nos modelos, todas as colunas são interpoladas para terem suas falhas preenchidas. 

A função de regressão linear está com o parâmetro *Normalize = True*

**Tutoriais usados:** [Kaggle](https://www.kaggle.com/shashankasubrahmanya/missing-data-imputation-using-regression), [Medium](https://medium.com/data-hackers/implementando-regress%C3%A3o-linear-simples-em-python-91df53b920a8)

**Documentação SciKit Learn** [link](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html?highlight=linearregression#sklearn.linear_model.LinearRegression)

**Pendências:**
* Calcular a eficiencia da regressão 
* Preencher as falhas por outros métodos e comparar
* Estudo de correlação para saber qual o melhor posto paa usar de preditor

In [ ]:
def regressao (df_y, df_x, colunas_y, colunas_x):

  '''A função retorna o dataframe com novas colunas com as falhas preenchidas por regressão
  \nOBS: Usar outros postos como variável preditora  
  \nOBS2: Realiza interpolação nas colunas antes do modelo de regressão

  ----
  Parâmetros:
  df_y: o dataframe que terá suas falhas preenchidas (precessora)
  df_x: o dataframe que servirá de variável preditora 
  colunas_y: colunas que possuem falhas
  colunas_x: colunas que serão usadas como preditora
  
  '''
  
  for coluna in colunas_y:

    ### 1 - IMPLEMENTANDO O MODELO

    # Criando novas colunas que terao as falhas preenchidas  
    df_y[coluna+'_R']=df_y[coluna]

    # Criando um dataframe com as variaveis do modelo
    df = pd.DataFrame(index=df_y.index)
    # variável alvo - interpolando as falhas
    df['y'] = df_y[coluna].interpolate() 
    # variável preditora - interpolando as falhas
    df[colunas_x] = df_x[colunas_x].interpolate()
    
    # Criando o modelo
    model = LinearRegression(normalize=True)
    # Definindo as variaveis 
    X = df[colunas_x]
    y = df['y']
    # Treinando o modelo
    model.fit(X,y)

    # Fazendo a previsão 
    predict = model.predict(X)
    df[coluna+'_R'] = predict
    # Preenchendo as falhas com as previsões do modelo
    df_y.loc[df_y[coluna].isnull(), coluna + '_R'] =  model.predict(X)[df_y[coluna].isnull()]

    # 2 - AVALIANDO O MODELO
    #print(model.score(X,y))

  return (df_y)

In [ ]:
def regressao2 (df_y, df_x, colunas_y, colunas_x):

  '''A função retorna o dataframe com novas colunas com as falhas preenchidas por regressão
  \nOBS: Usar outros postos como variável preditora  
  \nOBS2: Realiza preenchimento das falhas com `fillna()` antes do modelo de regressão

  ----
  Parâmetros:

  `df_y`: o dataframe que terá suas falhas preenchidas (precessora)
  `df_x`: o dataframe que servirá de variável preditora 
  `colunas_y`: colunas que possuem falhas
  `colunas_x`: colunas que serão usadas como preditora
  
  '''
##### CONJUNTO DE DADOS
  df = pd.DataFrame(index=df_y.index)

  Y = df_y[col_y]

  # Preenchimento da estação preditora com KNN
  X = df_x[col_X]
  imputer = KNNImputer()
  X = imputer.fit_transform(X)

  # Divisão dos dados 
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42)

##### IMPLEMENTANDO O MODELO DE REGRESSAO
      
  model = LinearRegression(normalize=True)

  # Treinando o modelo
  model.fit(X_train,y_train)

  # Fazendo a previsão 
  y_pred = model.predict(X_test)
  df[coluna+'_R'] = y_pred

##### ARMAZENANDO OS RESULTADOS

  # Criando novas colunas que terao as falhas preenchidas  
  df_y[coluna+'_R']=df_y[coluna]

  # Preenchendo as falhas com as previsões do modelo
  df_y.loc[df_y[coluna].isnull(), coluna + '_R'] =  model.predict(X)[df_y[coluna].isnull()]

##### AVALIANDO O MODELO
  print(model.score(X_test,y_pred))

  return (df_y)

Tutorial do Medium 
[link](https://medium.com/@Cambridge_Spark/tutorial-introduction-to-missing-data-imputation-4912b51c34eb)

In [ ]:
def regressao3 (df, col_X, col_y):

  '''Função para preenchimento de falhas. 
  O conjunto de treino consiste no df sem as falhas.
  Usa o KNN inputer para o conjunto de teste.

  Parâmetros:
  - df: dataframe com as falhas 
  - col_x: nome das colunas preditoras
  - col_y: nome das colunas que terão suas falhas preenchidas

  Atualizado
  '''

  ### DIVISAO DO CONJUNTO DE DADOS

  # Conjutno de Treinamento - Criar um subset sem as falhas 
  df_2 = pd.DataFrame(df)
  df_2 = df_2.dropna(subset = [col_y])
  df_2 = df_2.dropna(subset = col_X)
  
  X_train = df_2[col_X]
  y_train = df_2[col_y]

  # Conjunto de Teste - dataset completo com falhas preenchidas por KNN
  X_test = pd.DataFrame(df[col_X])
  imputer = KNNImputer()
  X_test = imputer.fit_transform(X_test)

  #X_test = X_test[col_X][y_falhas]

  # Criar subset somente com as falhas 
  y_falhas = df[col_y].isnull()
 
  ###IMPLEMENTANDO O MODELO

  # Criando o modelo
  model = LinearRegression(normalize=True)

  # Treinando o modelo
  model.fit(X_train,y_train)

  # Fazendo a previsão com dados de teste
  predict = model.predict(X_test)
  y_falhas['Previsão'] = predict

  ###ARMAZENANDO OS RESULTADOS

  # Criando novas colunas que terao as falhas preenchidas  
  df[str(col_y)+'_R']=df[col_y]

  # Preenchendo as falhas com as previsões do modelo
  df.loc[df[col_y].isnull(), str(col_y) + '_R'] =   predict[df[col_y].isnull()]

  #AVALIANDO O MODELO
  #model.score(X_train,y_train)

  return (df)

### **Teste**

In [ ]:
# piller_mensal_R = regressao3(piller_mensal, ['Maxima','NumDiasDeChuva'], 'Total') 
# piller_mensal_R['08-2019':'03-2020']

,Total,Maxima,NumDiasDeChuva,Total_R
Data,,,,
2019-08-31,52.2,7.4,13.0,52.200000
2019-09-30,93.8,18.4,13.0,93.800000
2019-10-31,NaN,NaN,NaN,186.278448
2019-11-30,NaN,NaN,NaN,241.629228
2019-12-31,NaN,NaN,NaN,296.980008
2020-01-31,301.1,32.8,27.0,301.100000
2020-02-29,222.9,26.3,25.0,222.900000
2020-03-31,250.0,36.3,20.0,250.000000


##**Função `KNN` - Preenchimento de Falhas**

A função retorna o DataFrame com novas colunas onde as falhas foram preenchidas pelo método *k-Nearest Neighbors*. 

**Parâmetros:**
* `df` DataFrame com as falhas
* `colunas_falhas` colunas com falhas - deve ser mais de uma coluna (?)

**Fonte:** [SciKit Learn](https://scikit-learn.org/stable/modules/impute.html#nearest-neighbors-imputation)

**Pendências:**
* avaliação de desempenho no modelo

In [ ]:
def KNN(df, colunas_falhas):

  '''A função retorna o DataFrame com novas colunas com as falhas preenchidas 
  pela algortimo de k-Nearest Neighbors
  '''

  for coluna in colunas_falhas:

    # Definindo o modelo
    model = KNNImputer()
    # Definindo a variável
    X = df[colunas_falhas]
    # Treinando e executando o modelo 
    previsao = model.fit_transform(X)

    # Criando um DataFrame com os resultados
    df_KNN=pd.DataFrame(data = previsao, index = df.index, columns = colunas_falhas )
    
    # Inserindo uma coluna com as falhas preenchidas
    df[coluna+'_KNN'] = df_KNN[coluna]

  return (df)

##Função `criando_dados_modelo` - Criação das variáveis dos modelo [Não usada]

Essa função retorna um dataframe com todas as variaveis que serão usadas no modelo de redes neurais, de acordo como especificações de entrada indicada no df `modelos_entradas`. 

A última coluna será a variável alvo, e as demais serão as variáveis preditoras


**Parâmetros**:
  - `modelos_entradas` dataframe que possui as opções de entrada que serão testadas
  - `num_modelo` o número do modelo que será criado
  - `df_pluvio` df e coluna que possui as dados pluviométricos médios
  - `df_vazaomontante` df e coluna com as vazões médias da estação que será usada
  como controle à montante
  - `df_vazao` df e coluna com as vazões médias que será a variáveis y

In [ ]:
def criando_dados_modelo2(modelos_entradas,num_modelo,df_entradas):

  '''Essa função retorna:
  - data_modelo: df com todos as colunas 
  - colunas_x: df com caracteristicas
  - colunas_y: df com variavel alvo

  A última coluna será a variável alvo, e as demais serão as variáveis x

  ----

  Parâmetros:
  - `modelos_entradas`: dataframe que possui as opções de entrada que serão testadas
  - `num_modelo`: o número do modelo que será criado
  - `df_entradas`: df contendo todas as caracteristicas disponiveis
  Atualizado2
  '''

  defasagem = modelos_entradas['Def'][num_modelo]
  data_modelo = pd.DataFrame()
  colunas_x = []

  # 1. CARACTERISTICAS

  for column in df_entradas:
    
    # Caso a caracteristica esteja no modelo
    if modelos_entradas.loc[num_modelo, column]:
      # Adicionando os dados em defasagem
      for i in range(1,defasagem+1):
        coluna = str(column) + ' T-' +str(i)
        data_modelo[coluna] = df_entradas[column].shift(-i)
        colunas_x.append(coluna)
     
  # 2. VARIAVEL ALVO
  data_modelo['Vazao_T'] = df_entradas['V_Galdi']

  # Excluindo as instancias com falhas que surgem quando fazemos a defasagem
  data_modelo.dropna(0,inplace=True)

  # Retornando algumas informações do modelo
  instancias, observacoes  = data_modelo.shape[0], data_modelo.size
  inicio, fim = data_modelo.index[0], data_modelo.index[instancias-1]

  modelos_entradas.loc[num_modelo,'Inicio'] = inicio
  modelos_entradas.loc[num_modelo,'Fim'] = fim
  modelos_entradas.loc[num_modelo,'Observacoes'] = observacoes

  # Coeficiente de Correlação de Pearson 
  corr = data_modelo.corr().mean().mean()
  modelos_entradas.loc[num_modelo,'Correlacao'] = corr

  # Retornando as colunas X e Y
  colunas_y = data_modelo['Vazao_T']
  colunas_x = data_modelo[colunas_x]

  print('O modelo %i possui %i observações, de %s a %s (%i meses).'%(num_modelo,
                                                          observacoes,
                                                          str(inicio),
                                                          str(fim),
                                                          instancias))
  return (data_modelo,colunas_x,colunas_y)

